### RECONOCIMIENTO FACIAL 🧑📸

#### Codigo basico de ejemplo para detectar caras con mediapipe

pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se pudo acceder a la cámara.")
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame_rgb)
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(frame_bgr, detection)
        cv2.imshow('Detección de Caras', frame_bgr)
        if cv2.waitKey(1) & 0xFF == ord('q'): # Cierra la ventana con la tecla 'q'
            break
cap.release()
cv2.destroyAllWindows()


#### Codigo de ejemplo para contectarnos a la bd, y comprobar que funciona
pip install mysql-connector-python


In [ ]:
import mysql.connector

db_connection = mysql.connector.connect(
    host="localhost",
    user="root",     
    password="root", 
    database="datadb"  
)

# Creamos un cursor para ejecutar sentencias SQL
cursor = db_connection.cursor()

# Leemos de la bd
with open("database.sql", "r", encoding="utf-8") as file:
    sql_script = file.read()

try:
    for statement in sql_script.split(';'):
        if statement.strip(): # Evitamos ejecutar líneas vacías
            cursor.execute(statement)
    
    # Guardamos los cambios
    db_connection.commit()
    print("El script SQL se ejecutó correctamente.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    db_connection.rollback()  # En caso de error, se revierten los cambios
    
cursor.close()
db_connection.close()


El script SQL se ejecutó correctamente.


Parte 1: Guardar los embeddings faciales en la base de datos

En esta primera parte, el objetivo es permitir que el usuario guarde las características faciales (embeddings) de su rostro en una base de datos MySQL. Para ello, el sistema detectará la cara del usuario utilizando el modelo de MediaPipe FaceMesh y, cuando el usuario presione la tecla "s", las características faciales serán extraídas y almacenadas en la base de datos.

SE TOMARON 10 PARA NO LLENAR MUCHO LA BD, SI QUIERES QUE EL MODELO SEA MEJOR AUMENTA EL NUMERO DE EMBEDDINGS Y YA CREO...

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import mysql.connector
import json

# Inicializamos MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

db = mysql.connector.connect(
    host="localhost",
    user="root", 
    password="root",
    database="datadb" 
)

cursor = db.cursor()

cap = cv2.VideoCapture(0)

# Inicializamos el modelo FaceMesh de MediaPipe
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5) as face_mesh:
    
    embeddings = []
    labels = []
    
    embeddings_guardados = 0
    NUMERO_MAXIMO_EMBEDDINGS = 5 
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se pudo acceder a la cámara.")
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb) 
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR) 

        if results.multi_face_landmarks:
            for i, face_landmarks in enumerate(results.multi_face_landmarks):
                mp_drawing.draw_landmarks(frame_bgr, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)

                face_coords = np.array([(landmark.x, landmark.y, landmark.z) for landmark in face_landmarks.landmark])

                # Normalizamos las coordenadas de la cara (centrar en el centro de la imagen)
                face_coords[:, 0] -= np.mean(face_coords[:, 0])  
                face_coords[:, 1] -= np.mean(face_coords[:, 1])  
                face_coords[:, 2] -= np.mean(face_coords[:, 2]) 
                
                # Calculamos un embedding
                embedding = np.mean(face_coords, axis=0)  

                if embeddings_guardados < NUMERO_MAXIMO_EMBEDDINGS:
                    if not any(np.array_equal(embedding, e) for e in embeddings):  
                        embeddings.append(embedding)
                        labels.append(f"Persona {i+1}")
                        embeddings_guardados += 1
                        print(f"Nuevo embedding de Persona {i+1} almacenado.")
        
        cv2.imshow('Detección de Caras', frame_bgr)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('s') and results.multi_face_landmarks:
            if embeddings_guardados > 0:
                for i, embedding in enumerate(embeddings):
                    embedding_json = json.dumps({"embedding": embedding.tolist()})  
                    nombre = f"Alejandro"  # Pon aquí tu nombre
                    query = "INSERT INTO mi_tabla (nombre, data) VALUES (%s, %s)"
                    cursor.execute(query, (nombre, embedding_json))
                    db.commit()
                    print(f"Embedding de {nombre} guardado en la base de datos.")

        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cursor.close()
    db.close()


Este script es para consultar el contenido de la bd de lo que hemos guardado en el anterior script

In [ ]:
import mysql.connector
import json
import numpy as np

db = mysql.connector.connect(
    host="localhost",
    user="root",  
    password="root",
    database="datadb"
)

cursor = db.cursor()

# Consultamos para obtener todos los registros
query = "SELECT id, nombre, data FROM mi_tabla"
cursor.execute(query)

# Leemos los registros
resultados = cursor.fetchall()

# Procesamos los resultados
embeddings = []
nombres = []

for row in resultados:
    id, nombre, data_json = row # Convertimos el JSON de vuelta a una lista de embeddings
    data = json.loads(data_json)  # Convertimos el JSON a un diccionario
    embedding = np.array(data["embedding"])  # Obtenemos el array de embedding
    embeddings.append(embedding)
    nombres.append(nombre)
    print(f"ID: {id}, Nombre: {nombre}, Embedding: {embedding}")

cursor.close()
db.close()


Parte 2: Reconocimiento facial con NearestNeighbors

En esta segunda parte, el sistema utiliza el modelo de NearestNeighbors para intentar reconocer el rostro de la persona que se muestra frente a la cámara. Al detectar la cara, el sistema compara los embeddings faciales extraídos con los previamente almacenados en la base de datos para identificar al individuo. Si se encuentra una coincidencia, se muestra el nombre de la persona; de lo contrario, se etiquetará como "Desconocido".

pip install scikit-learn

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import mysql.connector
import json
from sklearn.neighbors import NearestNeighbors
import warnings

# Esto es para ignorar warnings de la librería protobuf
warnings.filterwarnings("ignore", category=UserWarning, module="google.protobuf")

db = mysql.connector.connect(
    host="localhost",
    user="root",  
    password="root", 
    database="datadb"  
)

cursor = db.cursor()

# Obtenemos todos los registros de la base de datos
query = "SELECT id, nombre, data FROM mi_tabla"
cursor.execute(query)

# Leemos los registros
resultados = cursor.fetchall()

embeddings = []
nombres = []
UMBRAL_VECINO_CERCANO = 0.80  # Valor aproximado para reconocer a una persona

# Cargamos los embeddings y los nombres
for row in resultados:
    id, nombre, data_json = row
    data = json.loads(data_json)  # Convertimos el JSON de vuelta a un diccionario
    embedding = np.array(data["embedding"])
    embeddings.append(embedding)
    nombres.append(nombre)

embeddings = np.array(embeddings)  # Convertimos la lista de embeddings a un arreglo de numpy

# Inicializamos el modelo NearestNeighbors de sklearn para la búsqueda de vecinos más cercanos
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(embeddings)

# Inicializamos MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Abrimos la camara
cap = cv2.VideoCapture(0)

# Inicializamos el modelo FaceMesh de MediaPipe
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se pudo acceder a la cámara.")
            break

        # Conviertimos la imagen a RGB (MediaPipe requiere imágenes RGB)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)  # Procesamos la imagen para detectar mallas faciales
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

        # Dibujamos la malla de la cara en la imagen
        if results.multi_face_landmarks:
            for i, face_landmarks in enumerate(results.multi_face_landmarks):
                # Extraemos las coordenadas de los puntos de la malla de la cara
                face_coords = np.array([(landmark.x, landmark.y) for landmark in face_landmarks.landmark])

                # Encontramos los puntos de la cara para formar el rectángulo
                x_min = int(np.min(face_coords[:, 0]) * frame_bgr.shape[1])
                x_max = int(np.max(face_coords[:, 0]) * frame_bgr.shape[1])
                y_min = int(np.min(face_coords[:, 1]) * frame_bgr.shape[0])
                y_max = int(np.max(face_coords[:, 1]) * frame_bgr.shape[0])

                # Dibujamos un cuadro alrededor de la cara
                cv2.rectangle(frame_bgr, (x_min - 20, y_min - 20), (x_max + 20, y_max + 20), (0, 255, 0), 2)  # Rectángulo verde

                # Usamos una representación simple del embedding con solo 3 características
                # Por ejemplo, podríamos usar los valores x, y, z del primer punto de la cara
                embedding = np.array([face_coords[0, 0], face_coords[0, 1], face_coords[0, 0] - face_coords[0, 1]])  # Primer punto
                
                # Realizamos la consulta de vecino más cercano
                distances, indices = nbrs.kneighbors([embedding])
                # Si el vecino más cercano tiene una distancia pequeña, lo reconocemos
                if distances[0][0] < UMBRAL_VECINO_CERCANO:  
                    nombre_detectado = nombres[indices[0][0]]
                else:
                    nombre_detectado = "Desconocido"  

                # Dibujamos el nombre de la persona encima del cuadro
                cv2.putText(frame_bgr, nombre_detectado, (x_min - 20, y_min - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Reconocimiento de Caras', frame_bgr)

        # Cerramos la ventana con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

cursor.close()
db.close()


Si deseas borrar todo el contenido de la bd

In [126]:
import mysql.connector
#
# ESTE SCRIPT BORRA EL CONTENIDO LA BD TEN CUIDADO, SE CONSCIENTE DE QUE LO HACES POR BUENA RAZON
#
def borrar_contenido_bd():
    try:
        connection = mysql.connector.connect(
            host='localhost',        
            user='root',      
            password='root',
            database='datadb'         
        )

        if connection.is_connected():
            cursor = connection.cursor()
            query = "DELETE FROM mi_tabla"
            cursor.execute(query)
            connection.commit()
            print("Todos los registros han sido borrados exitosamente.")
    
    except mysql.connector.Error as err:
        print(f"Error al borrar los registros: {err}")
    
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("Conexión a la base de datos cerrada.")

borrar_contenido_bd() # BORRA EL CONTENIDO DE LA BD SI ES LO QUE DESEAS

Todos los registros han sido borrados exitosamente.
Conexión a la base de datos cerrada.


Si deseas borrar todos los embeddings de un usuario en particular de la bd

In [127]:
import mysql.connector
#
# ESTE SCRIPT BORRA EL CONTENIDO DE UN USUARIO LA BD TEN CUIDADO, SE CONSCIENTE DE QUE LO HACES POR BUENA RAZON
#
def borrar_desde_nombre(nombre):
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root', 
            password='root',
            database='datadb'
        )

        if connection.is_connected():
            cursor = connection.cursor()
            query = "DELETE FROM mi_tabla WHERE nombre >= %s"
            cursor.execute(query, (nombre,))  
            connection.commit()  
            print(f"Se han borrado todas las filas a partir del nombre '{nombre}'.")

    except mysql.connector.Error as err:
        print(f"Error al borrar las filas: {err}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("Conexión a la base de datos cerrada.")


borrar_desde_nombre("Alejandro") # PONER AQUI EL NOMBRE DE LA PERSONA QUE QUIERAS ELIMINAR SUS EMBEDDINGS

Se han borrado todas las filas a partir del nombre 'Alejandro'.
Conexión a la base de datos cerrada.
